In [1]:
import hashlib
from datetime import datetime
import ecdsa

In [2]:
def get_timestamp():
    date = datetime.now()
    return str(datetime.timestamp(date))

In [18]:
class Transaction:
    def __init__(self, from_address, to_address, amount):
        self.from_address = from_address
        self.to_address = to_address
        self.amount = amount
        self.signature = None
        
    def calculate_hash(self):
        concat = self.from_address + self.to_address + str(self.amount)
        return hashlib.sha256(concat.encode('utf-8')).hexdigest()

    def sign_transaction(self, signing_key):
        if signing_key.get_verifying_key().to_string().hex() != self.from_address:
            raise ValueError("Error. You can't sign transactions of other wallets")
        transaction_hash = self.calculate_hash()
        self.signature = signing_key.sign(bytes(transaction_hash.encode('utf-8')))
        
    def isValid(self):
        if self.from_address == None:
            return True                  # mining reward transaction
        if (self.signature == None) or (len(self.signature) == 0):
            raise ValueError('Error. No signature in this transaction!')
        verifying_key = ecdsa.VerifyingKey.from_string(bytes.fromhex(self.from_address), curve=ecdsa.SECP256k1)
        return verifying_key.verify(self.signature, bytes(self.calculate_hash().encode('utf-8'))) #True of False
        

class Block:
    def __init__(self, timestamp, data, previous_hash = ""):     #data contains transactions
        self.timestamp = timestamp
        self.previous_hash = previous_hash
        self.data = data
        self.nonce = 0
        self.block_hash = self.calculate_hash()

    def calculate_hash(self):
        concat = self.previous_hash + self.timestamp + str(self.data) + str(self.nonce)
        return hashlib.sha256(concat.encode('utf-8')).hexdigest()
        
    def mine_block(self, difficulity):
        while self.block_hash[:difficulity] != "0" * difficulity:
            self.nonce += 1
            self.block_hash = self.calculate_hash()
        
    def get_block_data(self):
        return {
            "timestamp":self.timestamp,
            "previous_hash":self.previous_hash,
            "data":self.data,
            "block_hash":self.block_hash,
            "nonce":self.nonce
        }
        
    def has_valid_transactions(self):
        for transaction in self.data:
            if not transaction.isValid():
                return False
        return True
        

In [19]:
class Blockchain:
    def __init__(self, difficulity=5):
        self.chain = []
        self.difficulity = difficulity
        self.chain.append(self.create_genesis_block())
        self.mining_reward = 100
        self.pending_transactions = []
    
    def create_genesis_block(self):
        genesis_block = Block(get_timestamp(), [Transaction("","",0)], "0")
        genesis_block.mine_block(self.difficulity)
        return genesis_block
        
    def get_latest_block(self):
        return self.chain[-1]

    def add_transaction(self, transaction):
        if transaction.from_address == None or transaction.to_address == None:
            raise ValueError("Error. Transaction must have a from and to addresses")
        if transaction.amount <=0:
            raise ValueError("Error. Transaction amount must be greater than 0")
        if not transaction.isValid():
            raise ValueError("Error. Can't add an invalid transaction")
        self.pending_transactions.append(transaction)
        
    def mine_pending_transactions(self, mining_reward_address):
        self.pending_transactions.append(Transaction(None, mining_reward_address, self.mining_reward))
        new_block = Block(get_timestamp(), self.pending_transactions)
        new_block.previous_hash = self.get_latest_block().block_hash
        new_block.mine_block(self.difficulity)
        self.chain.append(new_block)
        self.pending_transactions = []

    def get_balance(self, address):
        balance = 0
        for block in self.chain:
            for transaction in block.data:
                if transaction.to_address == address:
                    balance += transaction.amount
                elif transaction.from_address == address:
                    balance -= transaction.amount
        return balance
            

    def check_validity(self):
        for i in range(1, len(self.chain)):
            current = self.chain[i]
            previous = self.chain[i-1]
            if current.previous_hash != previous.block_hash:
                return False
            if current.block_hash != current.calculate_hash():
                return False
            if not current.has_valid_transactions():
                return False
            return True
            
            
        



In [33]:
sk = ecdsa.SigningKey.generate(curve=ecdsa.SECP256k1)
private_key = sk.to_string().hex()
vk = sk.get_verifying_key()
public_key = vk.to_string().hex()

In [21]:
c = Blockchain(3)

In [22]:
t = Transaction(public_key, "address", 50)
t.sign_transaction(sk)

In [23]:
c.add_transaction(t)

In [24]:
c.mine_pending_transactions("")

In [25]:
c.get_balance("address3")

0

In [26]:
c.check_validity()

True